#DATAMINING PROJECT V1.6
***From Baptiste Danichert & Ahmed Abdel Aziz***

**Last update**: deleting Stopwords

**Description:** You have noticed that to improve one’s skills in a new foreign language, it is important to read texts in that language. These texts have to be at the reader’s language level. However, it is difficult to find texts that are close to someone’s knowledge level (A1 to C2). You have decided to build a model for English speakers that predicts the difficulty of a French written text. This can be then used, e.g., in a recommendation system, to recommend texts, e.g, recent news articles that are appropriate for someone’s language level. If someone is at A1 French level, it is inappropriate to present a text at B2 level, as she won’t be able to understand it. Ideally, a text should have many known words and may have a few words that are unknown so that the person can improve.



# 1. Loading the training data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/BapDanSI/DataMiningProject/main/data/training_data.csv')
df_pred = pd.read_csv('https://raw.githubusercontent.com/BapDanSI/DataMiningProject/main/data/unlabelled_test_data.csv')

In [ ]:
df_pred.head()

,id,sentence
0,0,Nous dûmes nous excuser des propos que nous eû...
1,1,Vous ne pouvez pas savoir le plaisir que j'ai ...
2,2,"Et, paradoxalement, boire froid n'est pas la b..."
3,3,"Ce n'est pas étonnant, car c'est une saison my..."
4,4,"Le corps de Golo lui-même, d'une essence aussi..."


In [ ]:
df.head()

,id,sentence,difficulty
0,0,Les coûts kilométriques réels peuvent diverger...,C1
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1
2,2,Le test de niveau en français est sur le site ...,A1
3,3,Est-ce que ton mari est aussi de Boston?,A1
4,4,"Dans les écoles de commerce, dans les couloirs...",B1


# 2. Dataframe analysis


In [ ]:
df.shape

(4800, 3)

-> 4800 rows and 2 columns (excluding first column "id")

In [ ]:
df.isnull().sum()

id            0
sentence      0
difficulty    0
dtype: int64

-> no NAs

In [ ]:
df.duplicated(subset="sentence").value_counts()

False    4800
dtype: int64

-> no duplicate in the data

#3.  Baseline

In [ ]:
np.random.seed = 0

In [ ]:
base_rate = max(df.value_counts('difficulty'))/df.shape[0]
print('Base rate:', round(base_rate,4))

Base rate: 0.1694


# 4. Data Cleaning

In [ ]:
df['sentence'] = df['sentence'].str.lower()
df_pred['sentence'] = df_pred['sentence'].str.lower()

In [ ]:
df['sentence'] = df['sentence'].str.replace('[^\w\s]','')
df_pred['sentence'] = df_pred['sentence'].str.replace('[^\w\s]','')

<ipython-input-11-858d7f573432>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['sentence'] = df['sentence'].str.replace('[^\w\s]','')
<ipython-input-11-858d7f573432>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_pred['sentence'] = df_pred['sentence'].str.replace('[^\w\s]','')


In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!pip install -U wordstats
!python -m spacy download fr_core_news_sm
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stopwords
from wordstats import Word, common_words
from collections import Counter
import string
import nltk
import spacy
nlp = spacy.load("fr_core_news_sm")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.3 MB/s 
     |████████████████████████████████| 1.2 MB 46.9 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 36.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for wordstats: filename=wordstats-1.0.7-py3-none-any.whl size=3616363 sha256=91cc7406ceaa589c080387ef3c1e51849b47516b2655157b08f6d721ff7732c4
  Stored in directory: /root/.cache/pip/wheels/07/27/7d/24ade697c516ed02f369a90c5fec463286a6a67a299f75b711
  Created wheel for configobj: filename=configobj-5.0.6-py3-none-any.whl size=34527 sha256=cf37a91baf009d0d20ad21a75982647922a8983c80e89950d18e9ee7d891c2f5
  Stored in directory: /root/.cache/pip/wheels/e0/ea/e1/7b17c0af6f0228de33d82556dbad4fe823d90696fc50f2c81d
Successfully built wordstats configobj
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWar

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
spacy_stopwords = spacy.lang.fr.stop_words.STOP_WORDS
print('Number of stopwords: %d' % len(spacy_stopwords))

Number of stopwords: 507


In [ ]:
def remove_stopwords(text):
    doc = nlp(text)
    words = [chunk.lemma_ for chunk in doc.noun_chunks if chunk.lemma_ not in nlp.Defaults.stop_words]
    return " ".join(words)
df['sentence'] = df['sentence'].apply(remove_stopwords)
df_pred['sentence'] = df_pred['sentence'].apply(remove_stopwords)

# 5. Classification Algorithms

Dependent variable (y) is the column named "difficulty".
<br>We split the data into 80% training and 20% test set.


In [ ]:
y = df['difficulty']
X = df['sentence']
X_pred = df_pred['sentence']

### i. Logistic Regression.
We use the following parameters for the LogisticRegressionCV():

* cross-validation to 5 folds
* maximum interation to 1000
* random state to 0

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1, 1))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

classifier = LogisticRegressionCV(solver='lbfgs', cv=5, max_iter=1000, random_state=0)
pl = Pipeline([('vectorizer', tfidf),
                 ('classifier', classifier)])
pl.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier',
                 LogisticRegressionCV(cv=5, max_iter=1000, random_state=0))])

In [ ]:
def evaluate(true, pred):
    precision = precision_score(true, pred,average='macro')
    recall = recall_score(true, pred,average='macro')
    f1 = f1_score(true, pred,average='macro')
    print(f"CONFUSION MATRIX:\n{confusion_matrix(true, pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(true, pred):.4f}")
    print(f"CLASSIFICATION REPORT:\n\tPrecision: {precision:.4f}\n\tRecall: {recall:.4f}\n\tF1_Score: {f1:.4f}")

In [ ]:
y_pred = pl.predict(X_test)
evaluate(y_test, y_pred)

CONFUSION MATRIX:
[[88 28 23  7 12  3]
 [54 51 27  8 13 11]
 [25 29 52 17 12 25]
 [11 13 17 49 24 30]
 [17  2 21 36 59 38]
 [12 10  8 19 35 74]]
ACCURACY SCORE:
0.3885
CLASSIFICATION REPORT:
	Precision: 0.3850
	Recall: 0.3887
	F1_Score: 0.3843


In [ ]:
predict = pl.predict(X_pred)

In [ ]:
submission= pd.DataFrame()
submission['id']= df_pred.index
submission['difficulty'] = predict

In [ ]:
submission.to_csv("submission.csv", index=False)

### ii. kNNeighbours

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

grid = {'n_neighbors':np.arange(1,100),
        'p':np.arange(1,3),
        'weights':['uniform','distance']}

knn = KNeighborsClassifier()
classifier_knn = GridSearchCV(knn, grid, cv=5)

pl_knn = Pipeline([('vectorizer', tfidf),
                 ('classifier', classifier_knn)])

pl_knn.fit(X_train, y_train)

print("Hyperparameters:", classifier_knn.best_params_)

Hyperparameters: {'n_neighbors': 1, 'p': 1, 'weights': 'uniform'}


In [ ]:
y_knn_predict = pl_knn.predict(X_test)
evaluate(y_test, y_knn_predict)

CONFUSION MATRIX:
[[126  20  11   4   0   0]
 [119  35   9   1   0   0]
 [120  25  12   2   0   1]
 [118  11   6   8   0   1]
 [151   3   6   3   7   3]
 [134   8   2   2   3   9]]
ACCURACY SCORE:
0.2052
CLASSIFICATION REPORT:
	Precision: 0.4185
	Recall: 0.2040
	F1_Score: 0.1549


In [ ]:
knn_predict = pl_knn.predict(X_pred)
submission_knn= pd.DataFrame()
submission_knn['id']= df_pred.index
submission_knn['difficulty'] = knn_predict
submission_knn.to_csv("submissionknn.csv", index=False)

###  iii. Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
pl_dtc = Pipeline([('vectorizer', tfidf),
                 ('classifier', dtc)])
pl_dtc.fit(X_train, y_train)
y_pred_dtc = pl_dtc.predict(X_test)

evaluate(y_test, y_pred_dtc)

CONFUSION MATRIX:
[[82 21 30 14  7  7]
 [61 40 24 22  8  9]
 [28 28 58 24  7 15]
 [14 13 21 49 23 24]
 [10 19 35 40 28 41]
 [16 18 22 30 33 39]]
ACCURACY SCORE:
0.3083
CLASSIFICATION REPORT:
	Precision: 0.3014
	Recall: 0.3108
	F1_Score: 0.3011


In [ ]:
dtc_predict = pl_dtc.predict(X_pred)
submission_dtc= pd.DataFrame()
submission_dtc['id']= df_pred.index
submission_dtc['difficulty'] = dtc_predict
submission_dtc.to_csv("submissiondtc.csv", index=False)

### iv. Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
pl_rfc = Pipeline([('vectorizer', tfidf),
                 ('classifier', rfc)])

pl_rfc.fit(X_train, y_train)

y_pred_rfc = pl_rfc.predict(X_test)

evaluate(y_test, y_pred_rfc)

CONFUSION MATRIX:
[[97 30 23  8  1  2]
 [65 55 32  6  4  2]
 [29 45 53 12  4 17]
 [22 15 25 38 13 31]
 [13 18 28 35 44 35]
 [16 18 20 19 19 66]]
ACCURACY SCORE:
0.3677
CLASSIFICATION REPORT:
	Precision: 0.3781
	Recall: 0.3675
	F1_Score: 0.3611


In [ ]:
rfc_predict = pl_rfc.predict(X_pred)
submission_rfc= pd.DataFrame()
submission_rfc['id']= df_pred.index
submission_rfc['difficulty'] = rfc_predict
submission_rfc.to_csv("submissionrfc.csv", index=False)

### v. Support Vector Classifier

In [ ]:
from sklearn.svm import SVC

svc = SVC()
pl_svc = Pipeline([('vectorizer', tfidf),
                 ('classifier', svc)])

pl_svc.fit(X_train, y_train)

y_pred_svc = pl_svc.predict(X_test)

evaluate(y_test, y_pred_svc)

CONFUSION MATRIX:
[[83 25 30 14  7  2]
 [47 49 31 15 11 11]
 [14 26 59 26 12 23]
 [ 6  7 21 59 17 34]
 [ 3  3 24 47 52 44]
 [ 7 12  9 24 30 76]]
ACCURACY SCORE:
0.3937
CLASSIFICATION REPORT:
	Precision: 0.3969
	Recall: 0.3957
	F1_Score: 0.3921


In [ ]:
svc_predict = pl_svc.predict(X_pred)
submission_svc= pd.DataFrame()
submission_svc['id']= df_pred.index
submission_svc['difficulty'] = svc_predict
submission_svc.to_csv("submissionsvc.csv", index=False)

###vi. AdaBoostClassifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abc = AdaBoostClassifier()
pl_abc = Pipeline([('vectorizer', tfidf),
                 ('classifier', abc)])

pl_abc.fit(X_train, y_train)

y_pred_abc = pl_abc.predict(X_test)

evaluate(y_test, y_pred_abc)

CONFUSION MATRIX:
[[75 24 27 28  7  0]
 [58 32 26 38  3  7]
 [27 29 25 53 13 13]
 [13  4 19 47 34 27]
 [11  6 23 52 41 40]
 [15  4 18 34 30 57]]
ACCURACY SCORE:
0.2885
CLASSIFICATION REPORT:
	Precision: 0.2973
	Recall: 0.2902
	F1_Score: 0.2858


In [ ]:
from sklearn.neural_network import MLPClassifier

mlpc = MLPClassifier()
pl_mlpc = Pipeline([('vectorizer', tfidf),
                 ('classifier', mlpc)])

pl_mlpc.fit(X_train, y_train)

y_pred_mlpc = pl_mlpc.predict(X_test)

evaluate(y_test, y_pred_mlpc)

CONFUSION MATRIX:
[[92 28 24  9  6  2]
 [58 54 31 11  4  6]
 [22 48 54 14  9 13]
 [ 6 28 26 44 20 20]
 [ 9 22 30 36 53 23]
 [10 21 15 22 29 61]]
ACCURACY SCORE:
0.3729
CLASSIFICATION REPORT:
	Precision: 0.3809
	Recall: 0.3727
	F1_Score: 0.3722


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
mlpc_predict = pl_mlpc.predict(X_pred)
submission_mlpc= pd.DataFrame()
submission_mlpc['id']= df_pred.index
submission_mlpc['difficulty'] = mlpc_predict
submission_mlpc.to_csv("submissionmlpc.csv", index=False)

# 6. Comparing the models

In [1]:
import pandas as pd

df_comparison = pd.DataFrame(columns=['Model_Name', 'Base_Rate', 'Precisions','Recall','F1-Score', 'Accuracy'])


df_comparison['Model_Name'] = ['Logistic Reg', 'KNN', 'Tree', 'Random Forest', 'Support Vector', 'AdaBooster', 'MLPC']
df_comparison['Base_Rate'] = ['0.1694', '0.1694', '0.1694', '0.1694', '0.1694', '0.1694', '0.1694']
df_comparison['Precisions'] = ['0.3850', '0.4185', '0.3014','0.3781', '0.3969', '0.2973', '0.3809']
df_comparison['Recall'] = ['0.3887', '0.2040', '0.3108','0.3675', '0.3957', '0.2902', '0.3727']
df_comparison['F1-Score'] = ['0.3843', '0.1549', '0.3011','0.3611', '0.3921', '0.2858', '0.3722']
df_comparison['Accuracy'] = ['0.3885', '0.2052', '0.3083','0.3677', '0.3937', '0.2885', '0.3729']


df_comparison

,Model_Name,Base_Rate,Precisions,Recall,F1-Score,Accuracy
0,Logistic Reg,0.1694,0.3850,0.3887,0.3843,0.3885
1,KNN,0.1694,0.4185,0.2040,0.1549,0.2052
2,Tree,0.1694,0.3014,0.3108,0.3011,0.3083
3,Random Forest,0.1694,0.3781,0.3675,0.3611,0.3677
4,Support Vector,0.1694,0.3969,0.3957,0.3921,0.3937
5,AdaBooster,0.1694,0.2973,0.2902,0.2858,0.2885
6,MLPC,0.1694,0.3809,0.3727,0.3722,0.3729
